In [2]:
using HDF5;                     # for .hd5 file manipulation
using DifferentialEquations;    # Provides a variety of differential solvers
using LinearAlgebra: Diagonal;  # Efficient Diagonal matrixes
using Statistics;               # Let's get this fucking bread 
using DataFrames;               # For succinct data manipulation
using CSV;                      # For writing the data to prevent constant re-running
using ForwardDiff;              # For profiling the gradients 
using Gadfly, Fontconfig, Cairo;# For plotting

In [4]:
"""
Takes time series data and calculates the average of each year.
"""
function bin(time_series::Vector, solution_vector::Vector)::Vector
    local binned_solution = Vector(undef, 0);   # Setting a vector to hold the bins 
    local whole_times = @.floor(time_series);   # Creating a vector of discrete time.
    for whole_time in unique(whole_times)       # Looping over the unique elements discrete times 
        local indexes = findall(whole_times .== whole_time);        # Getting the indexes
        push!(binned_solution, mean(solution_vector[indexes]));   # Appending to binned_solution
    end
    return binned_solution
end

bin

In [5]:
"""
Reads the flux (amounts), production (projection) and reserviour contents from 
a .hd5 file with file_name. It returns the transfer operator and production 
projection 
"""
function read_hd5(file_name::String)::Tuple{Matrix{Float64}, Vector{Float64}}
    local hd5 = h5open(file_name);                      # Opening the HDF5 file
    local F = hd5["fluxes"][1:end, 1:end];              # Retrieving the flux matrix 
    local P = hd5["production coefficients"][1:end];    # Retrieving the projection of the production 
    local N = hd5["reservoir content"][1:end, 1:end];   # The C14 reserviour contents 
    close(hd5);                                         # Closing the file 

    local λ = Diagonal([log(2) / 5730 for i in 1:11]);          # Constructing the decay matrix
    F = transpose(F) ./ N;                                      # The proportion flux
    local TO = transpose(F) - Diagonal(vec(sum(F, dims=2))) - λ;# Construncting the transfer operator
    return TO, P                                           
end

read_hd5

In [6]:
"""
Passed a solver function runs the solver and returns the speed and binned data
"""
function run_solver(solver, ∇::Function, U0::Vector, p)::Vector
    local problem = ODEProblem(∇, U0, (760.0, 790.0), p);   # Creating the ODEProblem instance
    local solution = solve(problem, reltol = 1e-6, solver()); # Solving the ODE  
    local time = Array(solution.t);                         # Storing the time sampling 
    solution = Array(solution)[2, 1:end];   # Storing the solution for troposphere 
    solution = bin(time, solution);         # Getting the annual means
    return solution;                        # Binning the results into years 
end

run_solver

In [7]:
"""
Calculates the gradient using a χ² loss function. 
"""
function select_gradient_solver(solver, ∇::Function, u0::Vector{Float64}, parameters)
    function loss_function(params)  
        local solution = run_solver(solver, ∇, u0, params); #! Naming
        local ΔC14 = (solution .- u0[2]) ./ u0[2] .* 1000;  # Calculating ΔC14
        local miyake = DataFrame(CSV.File("Miyake12.csv")); # Reading Miyake data
        local ΔC14 .+= mean(miyake.d14c[1:4]);  # Calculates the ticktack offset
        local χ² = sum(((miyake.d14c .- ΔC14[1:28]) ./ miyake.sig_d14c) .^ 2);  # Calculating χ² 
        return -0.5 * χ²
    end
    return ForwardDiff.gradient(loss_function, parameters);
end


select_gradient_solver

In [8]:
"""
This function constructs a plot of the input data assuming that it has the fields created by the more generalised 
"""
function speed_vs_accuracy_plot(filename::String)
    local r = DataFrame(CSV.File(filename));    # Reading data into the workspace
    local datavisual = Gadfly.plot(# Plot object for manipultation 
        y=r.accuracy, x=r.time_mean, label=r.solver,    # Raw data 
        Guide.ylabel("Accuracy"), Guide.xlabel("Time"), # X, Y Labels  
        # ymin=r.accuracy - r.accuracy_var,   # Y lower error bar
        # ymax=r.accuracy + r.accuracy_var,   # Y upper error bar
        xmin=r.time_mean - r.time_var,  # X lower error bar
        xmax=r.time_mean + r.time_var,  # X upper error bar
        Geom.xerrorbar, Geom.point, Geom.label, # Geometries 
        Scale.x_log10, Scale.y_log10)  # Setting log scales 
    return datavisual
end

speed_vs_accuracy_plot

Instead of taking +- $\sigma$ you can manually set the tick of the error bars. Basically like a boxplot. I need to have better labels and units.

Normailised terms are required.

In [ ]:
"""
So this is highly experimental basically I am passing a function into the profile function, which takes only solver as an argument. This done using the wrapper method that I used earlier
"""
function profile(solvers::Vector, f::Function, ∇::Function, u0::Vector{Float64}, parameters::Vector, len::Integer)::DataFrame
    #! Instead of using a matrix for data I could use a vector and then test the restults of 
    #! @.median()
    local data = Matrix{Float64}(undef, length(solvers), len);    # Creating the storage Matrix 
    local t_mean = Vector{Float64}(undef, length(solvers));     # For the mean of the times
    local t_var = Vector{Float64}(undef, length(solvers));      # For the time varience 
    local results = DataFrame(solver = @.string(solvers));      # DataFrame of summary Statistics

    for (index, solver) in enumerate(solvers)           # Looping over the solvers 
        local time_sample = Vector{Float64}(undef, 10); # The different run times of each trial 
        for i in 1:10
            local timer = time();               # Starting a timer
            solution = f(solver, ∇, u0, parameters);       # Running the solver
            time_sample[i] = time() - timer;    # ending the timer 

            if i == 10                          # Storing final run
                data[index, 1:end] = solution;   # filling C14
            end
        end
        t_mean[index] = mean(time_sample[2:end]);   # Storing run time ignoring compile run.
        t_var[index] = var(time_sample[2:end]);     # Storing time error
    end 

    data = (data' .- median(data, dims=1)')';       # Calculating deviations from median 
    data = @.abs(data);                             # Absolute deviations
    results.accuracy = vec(mean(data, dims=2));     # Mean deviation from the median 
    results.accuracy_var = vec(var(data, dims=2));  # Calculating the RMSE error << is better 
    results.time_mean = t_mean;                     # Storing the mean run time
    results.time_var = t_var;                       # Storing the varience of the time sample 
    
    return results
end

In [15]:
TO, P = read_hd5("Guttler14.hd5");      # Reading the data into the scope 

params = Vector{Float64}(undef, 6); # Storing the model params 
params[1] = 7.044873503263437;      # The mean position of the sinusoid 
params[2] = 0.18;                   # The modulation of the sinusoid w. r. t the mean
params[3] = 11.0;                   # Setting period of the sinusoid 
params[4] = 1.25;                   # The phase shift of the sinusoid
params[5] = 120.05769867244142;     # The height of the super gaussian 
params[6] = 12.0;                   # Width of the super-gaussian 

production(t, params) = params[1] * (1 + params[2] * 
    sin(2 * π / params[3] * t + params[4])) +           # Sinusoidal production 
    params[5] * exp(- (params[6] * (t - 775)) ^ 16);    # Super Gaussian event
derivative(x, params, t) = vec(TO * x + production(t, params) * P);  # The derivative system 

u0 = TO \ (- params[1] * P);  # Equilibriating the system

burnproblem = ODEProblem(derivative, u0, (-360.0, 760.0), params);  # Burn in problem  
burnsolution = solve(burnproblem, reltol=1e-6).u[end];              # Running model 

solvers = [TRBDF2, BS3, Tsit5, Rosenbrock23, ROS34PW1a, QNDF1, ABDF2, ExplicitRK, DP5, TanYam7, Vern6]

#! so the hessian retruns a 6 by 6 array meaning that I get a dimension mismathch. 
# I need to somehow further generalise the profile function without breaking it on existing cases and without adding more argument passes. Perhaps a structure is what I need here. That way I can defined the shape of the arrays and whatnot be setting elements of the profile structure. This could work quite nicely actually.

# time_sample = Vector{Float64}(undef, 10);
# for i in 1:10
#     timer = time();
#     hessians = ForwardDiff.hessian(select_gradient_solver(BS3, derivative, burnsolution), params);
#     time_sample[i] = time() - timer;
# end

# CSV.write("hessian_profiles.csv", hessians);
# hessian_plot = speed_vs_accuracy_plot("hessian_profiles.csv");
# draw(PDF("hessian.pdf", 10cm, 10cm), hessian_plot);

# fgrads = profile(solvers, select_gradient_solver, derivative, burnsolution, params, 6); 
solvers = profile([BS3, Tsit5, DP5], run_solver, derivative, burnsolution, params);

#* I think the process is complete now I need to test it with the others and implement the plot.

# CSV.write("gradient_profiles.csv", fgrads); # Writing the data to a CSV
# CSV.write("solver_profiles.csv", solvers);  # Writing solver profile to CSV

# fgrad_plot = speed_vs_accuracy_plot("gradient_profiles.csv");   # Plot objects 
# solver_plot = speed_vs_accuracy_plot("solver_profiles.csv");    # Plot object

# draw(PDF("forwards_mode_gradient.pdf", 10cm, 10cm), fgrad_plot);
# draw(PDF("solver.pdf", 10cm, 10cm), solver_plot);

LoadError: DimensionMismatch("arrays could not be broadcast to a common size; got a dimension with lengths 3 and 31")

In [12]:
solvers[2]

31×1×1 Array{Float64, 3}:
[:, :, 1] =
 709.9903784248754
 710.561611623547
 710.7344690152225
 710.5555966699058
 710.063799134401
 709.4277958025474
 708.8354542743984
 708.4831380362316
 708.476409350554
 708.807369330522
 709.3963505462355
 710.0678139156195
 710.5572436264085
   ⋮
 716.5444710898129
 716.3013610174344
 716.3419936861244
 716.484476792228
 716.5439724656616
 716.3550966979514
 715.8603409109899
 715.109769559736
 714.2461245215875
 713.4604483958059
 712.9288462576691
 712.7743900346481

In [16]:
"""
So this is highly experimental basically I am passing a function into the profile function, which takes only solver as an argument. This done using the wrapper method that I used earlier
"""
function profile(solvers::Vector, f::Function, ∇::Function, u0::Vector{Float64}, parameters::Vector)
    local data = Vector{Any}(undef, length(solvers));       # Creating the storage Matrix 
    local t_mean = Vector{Float64}(undef, length(solvers)); # For the mean of the times
    local t_var = Vector{Float64}(undef, length(solvers));  # For the time varience 
    # local results = DataFrame(solver = @.string(solvers));  # DataFrame of summary Statistics

    for (index, solver) in enumerate(solvers)           # Looping over the solvers 
        local time_sample = Vector{Float64}(undef, 10); # The different run times of each trial 
        for i in 1:10
            local timer = time();                       # Starting a timer
            solution = f(solver, ∇, u0, parameters);    # Running the solver
            time_sample[i] = time() - timer;            # ending the timer 

            if i == 10                  # Storing final run
                data[index] = solution; # filling C14
            end
        end
        t_mean[index] = mean(time_sample[2:end]);   # Storing run time ignoring compile run.
        t_var[index] = var(time_sample[2:end]);     # Storing time error
    end 

    #? So this is looking very promising but I am not sure that it works yet
    data = cat(data..., dims=3);                # Using multidimensional matrixes to store results
    meds = median(data, dims=3);                # Calculating the median along the third dimensions
    data = PermutedDimsArray(data, (3, 2, 1));  # Switching the dimensions for subtraction
    devs = @.abs(data .- meds);                 # Calculating the deviations
    devs = devs ./ data;                        # Normailised deviations,

    results.accuracy = vec(mean(devs, dims=1));     # Mean deviation from the median 
    results.accuracy_var = vec(var(devs, dims=1));  # Calculating the RMSE error  
    results.time_mean = t_mean;                     # Storing the mean run time
    results.time_var = t_var;                       # Storing the varience of time sample 
    
    return results
end

LoadError: UndefVarError: f not defined